# Knowledge and Data: Practical Assignment 2
## Manipulate local and external RDF Knowledge Graphs 

YOUR NAME: Kevin Schönhage

YOUR VUNetID: OMN496

*(If you do not provide your name and VUNetID we will not accept your submission).*

### Learning objectives

At the end of this exercise you should be able to perform some simple manipulations of RDF Data using the rdflib library. You should be able to: 

1. Add and retrieve information from a local RDF database
2. Represent RDF data in other formats, such as the .dot format for graph visualisation
3. Retrieve information from an RDF database created from Web Data
4. Query information from the Web with SPARQL

### Practicalities

Follow this Notebook step-by-step. 

Of course, you can do the exercises in any Programming Editor of your liking. 
But you do not have to. Feel free to simply write code in the Notebook. When 
everything is filled in and works, save the Notebook and submit it 
as a Jupyter Notebook, i.e. with an .ipynb extension. Please use as name of the 
Notebook your studentID+Assignment2.ipynb.  

We will not evaluate the programming style of your solutions. Yet we do look whether your solutions suggests an understanding, and whether they yield the correct output.

Note that all notebooks will automatically be checked for plagiarism: while similar answers can be expected, it is not allowed to directly copy the solutions from fellow students or TAs, or from the examples discussed during the lectures. Similarly, sharing your solutions with your peers is not allowed.

# A. Tasks related to local RDF Knowledge Graphs

This first cell will open a file 'example-from-slide.ttl' using the rdflib library. The first Practical Assignment should have taught you that manipulating symbols as strings is a major pain. 

Programming libraries, such as **rdflib**, help you with this mess once and for all, by parsing the files, creating appropriate datastructures (Graph()) and providing useful functions (such as serialize(), save() and much more). 
Check the website of rdflib http://rdflib.readthedocs.io/: this library does most of the hard work for you.

In [3]:
# Before starting with the tasks of this assignment, do not forget to install **rdflib** so we can start using it. 
%pip install rdflib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.1/565.1 kB 3.3 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [rdflib]2m1/2 [rdflib]
Note: you may need to restart the kernel to use updated packages.


In [6]:
from rdflib import Graph, RDF, Namespace, Literal, URIRef

g = Graph()

EX = Namespace('http://example.com/kad0/')
g.bind('ex',EX)

def serialize_graph():
    # g.serialize() returns a string
    print(g.serialize(format='turtle'))

def save_graph(filename):
    with open(filename, 'w') as f:
        g.serialize(f, format='nt')
        
def load_graph(filename):
    with open(filename, 'r') as f:
        g.parse(f, format='turtle')   

The file 'example-from-slides.ttl' formalises the knowledge base from the slides from Module 1, and a bit more. 

Here is how it looks when you load it into your program and serialise it with rdflib in turtle. 

In [7]:
load_graph('example-from-slides.ttl')
serialize_graph()

@prefix ex1: <http://example.com/kad/> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

ex1:Germany a ex1:EuropeanCountry .

ex1:Netherlands a ex1:Country ;
    ex1:hasCapital ex1:Amsterdam ;
    ex1:hasName "The Netherlands" ;
    ex1:neighbours ex1:Belgium .

ex1:hasCapital rdfs:range ex1:Capital ;
    rdfs:subPropertyOf ex1:containsCity .

ex1:Amsterdam a ex1:Capital .

ex1:Belgium a ex1:Country .

ex1:EuropeanCountry rdfs:subClassOf ex1:Country .

ex1:containsCity rdfs:domain ex1:Country ;
    rdfs:range ex1:City .

ex1:Capital rdfs:subClassOf ex1:City .




Now, we can manipulate the graph very easily, e.g. like in the following very simple function, which returns the predicate(s) that relate a subject to a literal object: 

In [8]:
for s,p,o in g:
    if type(o) is Literal:
        print(p)

http://example.com/kad/hasName


### - Task 1: (10 Points) Add information to an RDF graph

Add triples to the knowledge graph. Make sure that they have the right namespaces. 

Similarily to the triples already present in the file 'example-from-slides.ttl':
- add at least three new countries with their name and capital 
- add at least one triple with the `neighbour` predicate

Check: http://rdflib.readthedocs.io/en/stable/intro_to_creating_rdf.html

Remember that ```a``` is Turtle shorthand for ```rdf:type```.

In [9]:
ex = Namespace("http://example.com/kad/")
owl = Namespace("http://www.w3.org/2002/07/owl#")
rdf = Namespace("http://www.w3.org/1999/02/22-rdf-syntax-ns#")
rdfs = Namespace("http://www.w3.org/2000/01/rdf-schema#")

def add_country(country_label, capital_label):
    country = ex[country_label]
    capital = ex[capital_label]

    g.add((country, rdf.type, ex.Country))
    g.add((capital, rdf.type, ex.Capital))

    g.add((country, ex.hasName, Literal(country_label)))
    g.add((capital, ex.hasName, Literal(capital_label)))
    g.add((country, ex.hasCapital, capital))

add_country("France", "Paris")
add_country("Spain", "Madrid")
add_country("Italy", "Rome")

g.add((ex.Spain, ex.neighbours, ex.France))
g.add((ex.Italy, ex.neighbours, ex.France))

serialize_graph()

@prefix ex1: <http://example.com/kad/> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

ex1:Germany a ex1:EuropeanCountry .

ex1:Italy a ex1:Country ;
    ex1:hasCapital ex1:Rome ;
    ex1:hasName "Italy" ;
    ex1:neighbours ex1:France .

ex1:Netherlands a ex1:Country ;
    ex1:hasCapital ex1:Amsterdam ;
    ex1:hasName "The Netherlands" ;
    ex1:neighbours ex1:Belgium .

ex1:Spain a ex1:Country ;
    ex1:hasCapital ex1:Madrid ;
    ex1:hasName "Spain" ;
    ex1:neighbours ex1:France .

ex1:hasCapital rdfs:range ex1:Capital ;
    rdfs:subPropertyOf ex1:containsCity .

ex1:Amsterdam a ex1:Capital .

ex1:Belgium a ex1:Country .

ex1:EuropeanCountry rdfs:subClassOf ex1:Country .

ex1:Madrid a ex1:Capital ;
    ex1:hasName "Madrid" .

ex1:Paris a ex1:Capital ;
    ex1:hasName "Paris" .

ex1:Rome a ex1:Capital ;
    ex1:hasName "Rome" .

ex1:containsCity rdfs:domain ex1:Country ;
    rdfs:range ex1:City .

ex1:France a ex1:Country ;
    ex1:hasCapital ex1:Paris ;
    ex1:hasName

*After you ran the previous code (adding triples) the next cells will be executed on your extended graph. That is ok.*

### - Task 2a: (10 Points) Get unstructured information from an RDF graph (all Literals)

Use the functions available in the RDFLib library. Write a small function to print all Literals. 

Hint: there is a function in rdflib to test the type of an object (check previous examples in this notebook)

In [10]:
for s,p,o in g:
    
    if isinstance(o, Literal):
        print(o)
        
    pass

Paris
The Netherlands
Italy
Rome
Madrid
Spain
France


### - Task 2b: (10 Points) Get structured information from an RDF graph (all unique Predicates)

Please provide another function that gives a **unique** list of the predicates, ordered by occurrence (most occurring first). Print the number of occurences next to the predicate. The answer will look similar to this: 
<br/>7: http://www.w3.org/1999/02/22-rdf-syntax-ns#type
<br/>4: http://example.com/kad/hasName
<br/>4: http://example.com/kad/hasCapital
<br/>2: http://www.w3.org/2000/01/rdf-schema#subClassOf
<br/>2: http://www.w3.org/2000/01/rdf-schema#range
<br/>2: http://example.com/kad/neighbours
<br/>1: http://www.w3.org/2000/01/rdf-schema#subPropertyOf
<br/>1: http://www.w3.org/2000/01/rdf-schema#domain

In [11]:
for s,p,o in g:
    counts = {}

for s, p, o in g:
    counts[p] = counts.get(p, 0) + 1

for p, c in sorted(counts.items(), key=lambda x: (-x[1], str(x[0]))):
    print(f"{c}\t{p}")

    pass

10	http://www.w3.org/1999/02/22-rdf-syntax-ns#type
7	http://example.com/kad/hasName
4	http://example.com/kad/hasCapital
3	http://example.com/kad/neighbours
2	http://www.w3.org/2000/01/rdf-schema#range
2	http://www.w3.org/2000/01/rdf-schema#subClassOf
1	http://www.w3.org/2000/01/rdf-schema#domain
1	http://www.w3.org/2000/01/rdf-schema#subPropertyOf


# B. Tasks related to Graph visualisations 

### - Task 3a: (20 Points) From RDF to .dot 


In the lecture, we have seen two ways of writing a knowledge graph (simple n-triples, and simple turtle). Let us consider a 3rd syntax, this time a syntax that is useful for visualisation. One standard for visualising graphs is the .dot format.

Print the knowledge graph in .dot file format. Check https://graphviz.gitlab.io/documentation/ and https://graphviz.readthedocs.io/en/stable/ for the documentation. You will only need very little of this information, and the most relevant information can be found in the examples that are given. 

<br>Basically, an RDF graph in .dot format starts with `digraph G {` and then a list of links of the following form:
<br>`s -> o [label="p"]`
<br>for every triple in the graph (separated by `;`). Do not forget to end with a closing bracket. `}`

An example is 
     
     digraph G { s1 -> o1 [label="p1"] ; s2 -> o2 [label="p2"] } 
     
for an RDF graph [(s1 p1 o1), (s2 p2 o2)]

In [12]:
# install and import the graphviz library
%pip install graphviz
import graphviz

from IPython import display

Note: you may need to restart the kernel to use updated packages.


First, create an auxiliary function which strips the namespaces from URIs. This is necessary to make the node names readable when visualizing the .dot graph. Make sure that literals are enclosed by quotation marks. Hint: use `'"..."'` or `"\"...\""` to insert quotation marks in Python strings.

In [17]:
def strip(e):
    if isinstance(e, URIRef):
        s = str(e)
        s = s.rsplit('#', 1)[-1]
        s = s.rsplit('/', 1)[-1]
        return s
    return str(e)
    
    pass

Next, convert your graph to the .dot format.

In [18]:
dot = graphviz.Digraph(strict=True, graph_attr={"dpi":"50"})
for s, p, o in g:
    s_label = strip(s)
    p_label = strip(p)

    if isinstance(o, Literal):
        o_id = f"lit_{abs(hash(str(o))) % 10**8}"
        dot.node(s_label)
        dot.node(o_id, label=str(o), shape="plaintext")
        dot.edge(s_label, o_id, label=p_label)
    else:
        o_label = strip(o)
        dot.edge(s_label, o_label, label=p_label)

    pass

View the end result as .dot syntax and as a graph:

In [19]:
print(dot.source)

#[Didnt work so I put it in comment --> display.display_svg(dot)  # paste the source at www.webgraphviz.com if this does not produce anything 

strict digraph {
	graph [dpi=50]
	Paris
	lit_40346647 [label=Paris shape=plaintext]
	Paris -> lit_40346647 [label=hasName]
	containsCity -> City [label=range]
	Italy -> Country [label=type]
	hasCapital -> containsCity [label=subPropertyOf]
	Netherlands
	lit_16578088 [label="The Netherlands" shape=plaintext]
	Netherlands -> lit_16578088 [label=hasName]
	France -> Country [label=type]
	EuropeanCountry -> Country [label=subClassOf]
	Italy
	lit_42111142 [label=Italy shape=plaintext]
	Italy -> lit_42111142 [label=hasName]
	containsCity -> Country [label=domain]
	Rome
	lit_31283971 [label=Rome shape=plaintext]
	Rome -> lit_31283971 [label=hasName]
	Spain -> Country [label=type]
	Germany -> EuropeanCountry [label=type]
	Netherlands -> Amsterdam [label=hasCapital]
	Italy -> Rome [label=hasCapital]
	Paris -> Capital [label=type]
	Netherlands -> Belgium [label=neighbours]
	Madrid
	lit_15576347 [label=Madrid shape=plaintext]
	Madrid -> lit_15576347 [label=hasName]
	Amsterdam -> Capital [label=typ

### - Task 3b: (10 Points) From RDF to .dot with "semantic information"

There is a conceptual distinction between properties, instances and classes (sets of instances). A simple way of checking is the following

1. in a triple (s a o), with predicate a (which is a special abbreviation for the predicate rdf:type), the s is an Instance, and o is a Class. 
2. in a triple (s rdfs:subClassOf o) both s and o are Classes. 
3. in a triple (p rdfs:domain o) p is a Property and o is a Class. 
4. in a triple (p rdfs:range o)  p is a Property and o is a Class. 

Update the .dot representation for an RDF graph that:

- renders all predicates that are defined in the RDF namespace as dotted lines,
- renders all classes as rectangles,
- renders all literals as plain text (no enclosure), and
- renders all instances with the color blue. 

Check how your graph looks once finished. Hint: you can use the `color`, `shape` and `style` attributes in the node and edge function (see the documentation).

In [20]:
for s, p, o in g:

    if isinstance(o, Literal):                     # literals als platte tekst
        o_label = strip(o)
        dot.node(o_label, shape="plaintext")
        dot.edge(s_label, o_label, label=p_label)

    elif p == rdfs.subClassOf:                     # Class ⊑ Class (beide box) + gestippelde rand
        o_label = strip(o)
        dot.node(s_label, shape="box")
        dot.node(o_label, shape="box")
        dot.edge(s_label, o_label, label=p_label, style="dotted")

    elif p == rdf.type:                            # instance a Class (Class in box) + gestippelde rand
        o_label = strip(o)
        dot.node(o_label, shape="box")
        dot.edge(s_label, o_label, label=p_label, style="dotted")

    else:                                          # overige triples normaal
        o_lbl = strip(o)
        dot.edge(s_label, o_label, label=p_label)

print(dot.source)

strict digraph {
	graph [dpi=50]
	Paris
	lit_40346647 [label=Paris shape=plaintext]
	Paris -> lit_40346647 [label=hasName]
	containsCity -> City [label=range]
	Italy -> Country [label=type]
	hasCapital -> containsCity [label=subPropertyOf]
	Netherlands
	lit_16578088 [label="The Netherlands" shape=plaintext]
	Netherlands -> lit_16578088 [label=hasName]
	France -> Country [label=type]
	EuropeanCountry -> Country [label=subClassOf]
	Italy
	lit_42111142 [label=Italy shape=plaintext]
	Italy -> lit_42111142 [label=hasName]
	containsCity -> Country [label=domain]
	Rome
	lit_31283971 [label=Rome shape=plaintext]
	Rome -> lit_31283971 [label=hasName]
	Spain -> Country [label=type]
	Germany -> EuropeanCountry [label=type]
	Netherlands -> Amsterdam [label=hasCapital]
	Italy -> Rome [label=hasCapital]
	Paris -> Capital [label=type]
	Netherlands -> Belgium [label=neighbours]
	Madrid
	lit_15576347 [label=Madrid shape=plaintext]
	Madrid -> lit_15576347 [label=hasName]
	Amsterdam -> Capital [label=typ

### - Task 4: (10 Points) Deriving implicit knowledge (a bit of schema)

We will look into Schema information in the latter modules, but let us try already to find some implicit information in a first bit of inferencing: whenever there are two statements (s rdf:type o) and (o rdfs:subClassOf o2) we can derive (and later prove) that (s rdf:type o2). 

Write a procedure that adds all implied triples to our knowledge graph, and which prints each implied triple.

In [21]:
for s, _, o in g.triples((None, rdf.type, None)):
    for _, _, o2 in g.triples((o, rdfs.subClassOf, None)):
        t = (s, rdf.type, o2)
        if t not in g:
            g.add(t)
            print(t)

(rdflib.term.URIRef('http://example.com/kad/Amsterdam'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://example.com/kad/City'))
(rdflib.term.URIRef('http://example.com/kad/Paris'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://example.com/kad/City'))
(rdflib.term.URIRef('http://example.com/kad/Madrid'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://example.com/kad/City'))
(rdflib.term.URIRef('http://example.com/kad/Rome'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://example.com/kad/City'))
(rdflib.term.URIRef('http://example.com/kad/Germany'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://example.com/kad/Country'))


# C. Tasks related to local copies of external RDF Datasets using SPARQL

Until now, we have manipulated local knowledge graphs, but as we claimed in the lectures, the advantage of knowledge graphs is that they can easily be linked with other datasets on the Web. 

In the remaining 3 tasks, we will manipulate data from the Web, and ask complex queries over this web data. 

In the first task, we will access web data, make a local copy of it, and then query it. In the other two tasks, we will query live data directly from web Knowledge Graphs (in this case, the SPARQL endpoint of DBPedia). 

### - Task 5: (10 Points) Show and manipulate data about RDF resources on the Web 

With rdflib we can easily load a local graph, but we can just as well retrieve a graph from the Web. Here, we will do so using the *requests* library, which allows us to fire a request to any server and/or SPARQL endpoint and to capture the response. The following snippet does so for the resource Netherlands from Dbpedia, by using the 'DESCRIBE' keyword to give us all triples about The Netherlands, and then loads it in a RDFlib Graph object. Note that, in the next assignment, we will learn a more high-level approach that hides most of the raw request details.

Should, for some external reason, the code below fail to retrieve the graph from DBpedia, then you can manually load the turtle file `dbpedia_AmsterdamBelgium.ttl` which is located in this directory and which contains the same triples. Only do this as a last resort!

In [23]:
# install the library
%pip install requests

Note: you may need to restart the kernel to use updated packages.


In [30]:
import requests

endpoint = "https://dbpedia.org/sparql"
query = 'DESCRIBE <http://dbpedia.org/resource/Netherlands>'

payload = {'query':query, 'format':'text/turtle'}
response = requests.post(endpoint, data = payload)

g = Graph()
g.parse(data=response.text, format='ttl')

<Graph identifier=N74642cf7dd044400a551673249cc261b (<class 'rdflib.graph.Graph'>)>

Now do the same for Belgium

In [31]:
query = 'DESCRIBE <http://dbpedia.org/resource/Belgium>'

payload = {'query': query, 'format':'text/turtle'}
response = requests.post(endpoint, data = payload)

g.parse(data=response.text, format='ttl')   # merging the graphs


payload = {'query':query, 'format':'text/turtle'}
response = requests.post(endpoint, data = payload)

g.parse(data=response.text, format='ttl')  # calling parse again merges the graphs

<Graph identifier=N74642cf7dd044400a551673249cc261b (<class 'rdflib.graph.Graph'>)>

Let us start by showing diverse bits of information w.r.t  The Netherlands and Belgium in DBPedia. It is very similar to task 1, but now with Web Data. 

First, query the graph g (now containing the DBPedia information about both countries) and check which motor ways cross both countries.

In [32]:
qres = g.query(
   """
    PREFIX dbr: <http://dbpedia.org/resource/>
    PREFIX dbo: <http://dbpedia.org/ontology/>
    SELECT ?s
        WHERE {
            ?s a dbo:Road ;
               dbo:county dbr:Netherlands ;
               dbo:county dbr:Belgium .
        }
        LIMIT 10
       """)
for row in qres:
    print("%s" % row)

Write a query to check whether you can find someone who was born in The Netherlands and died in Belgium? You need to look at the data to know which property you should check for. 

To get an intuition of what is in the knowledge graph you might want to look at the human readable rendering on : http://dbpedia.org/resource/Netherlands

In [33]:
# Your code here
qres = g.query(
    """
    PREFIX dbr: <http://dbpedia.org/resource/>
    PREFIX dbo: <http://dbpedia.org/ontology/>
    SELECT ?s
        WHERE {
            ?s a dbo:River ;
                dbo:country dbr:Netherlands ;
                dbo:country dbr:Belgium .
        }
        LIMIT 10
        """)
for row in qres:
    print("%s" % row)


### - Task 6: (20 Points) Ask SPARQL against live data using Yasgui

Yasgui (https://yasgui.triply.cc) is a nice graphical interface for asking queries.

Run a new query against http://dbpedia.org/sparql that does the following:

- Find all languages spoken in countries that are not official languages of that country.
- The query should return two colums: the country, and the number of languages.
- Order the countries by the number of unofficial languages, from high to low.

Note that
- Countries are assumed to be of type `dbo:Country`
- Languages are assumed to be of type `dbo:Language` (and can be linked by many different predicates)
- Official languages are linked using predicate `dbo:officialLanguage`
- Not all countries have an official language (in which case we return all linked languages)

Hint: there is more than one possible solution. See https://www.w3.org/TR/sparql11-query/ for documentation on SPARQL.

In [36]:
'''
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbo: <http://dbpedia.org/ontology/>

SELECT ?country (COUNT(DISTINCT ?lang) AS ?numUnofficial)
WHERE {
  ?country a dbo:Country .
  ?country ?p ?lang .
  ?lang a dbo:Language .

  FILTER NOT EXISTS { ?country dbo:officialLanguage ?lang }
}
GROUP BY ?country
ORDER BY DESC(?numUnofficial)
'''

'\nPREFIX dbr: <http://dbpedia.org/resource/>\nPREFIX dbo: <http://dbpedia.org/ontology/>\n\nSELECT ?country (COUNT(DISTINCT ?lang) AS ?numUnofficial)\nWHERE {\n  ?country a dbo:Country .\n  ?country ?p ?lang .\n  ?lang a dbo:Language .\n\n  FILTER NOT EXISTS { ?country dbo:officialLanguage ?lang }\n}\nGROUP BY ?country\nORDER BY DESC(?numUnofficial)\n'